# CS109b Final Project

# Milestone 3


by Danqing Wang, Wenshan Zheng, Zecai Liang


---

# Part 1. Feature Engineering

### Variable Group 1: ['year', 'rating', 'votes', 'popularity_TMDB', 'runtime_TMDB']
- variable ['year', 'rating', 'votes'] have missing values as 'NaN': we impute the missing value in training and test data with the mean value of training data
- [popularity_TMDB', 'runtime_TMDB'] have missing values as 0: we impute the missing value in training and test data with the mean value of training data

### Variables Group 2: ['title', 'plot', 'plot outline', 'overview_TMDB', 'tagline_TMDB']
- we combine all the text information for one movie, split the paragraph of string into bag-of-words, and return the top 10 PCs
- we used the PCA model trained from the training data to transform the test data

### Variable Group 3: ['mpaa']
- we extracted the text as the reason for 'mpaa', and applied similar text analysis as variable group 2, and return return the top 10 PCs

### Variable Group 4: ['director', 'cast', 'production company', 'writer']
- @ Danqing

### Variables Group 5: ['animation department', 'original music']
- we extract the number of person ID for both department, representing the number of staff working there

### Responses
@ Wehshan
- choose one between 'music' and 'musical'
- delete 'TY-movie'
- combined some genres (details and reason)

In [31]:
import pandas as pd

df_feature = pd.read_csv("feature_final.csv", index_col = 0)
df_genre = pd.read_csv("genre_final.csv", index_col = 0)

In [63]:
# x_train, x_test, y_train, y_test = generate_train_test_data(feature_data = df_feature, genre_data = df_genre, 
                                                             n_sample = 5000, train_ratio = 0.5, ran_state = 0)

In [69]:
# x_train.to_csv("x_train.csv")
# x_test.to_csv("x_test.csv")
# y_train.to_csv("y_train.csv")
# y_test.to_csv("y_test.csv")

### Run functions in the supplement before running this function

In [62]:
###### Function to generate train and test data for classification models #######
### ------ Input ---------- ###
# feature_data: a data frame with original features from IMDB and TMDB
# genre_data: a data grame with genres (already merged turned into one-hot coding), 
#             matching the feature_data by 'imdb_ids'
# n_sample: the number of data points to sample from the input data frame to fit model
# train_ratio: the percentage of training data among sampled data
# ran_state: seed for random sampling


def generate_train_test_data(feature_data, genre_data, n_sample = 5000, train_ratio = 0.5, ran_state = 0):
    
    import pandas as pd
    import numpy as np
    import re
    import random
    import warnings
    warnings.filterwarnings("ignore")

    # ------------------------------------------------------------------------------------------------------ #     

    ### sample data ###
    
    data_x = feature_data.sample(n = n_sample, random_state = ran_state)
    data_y = genre_data.ix[data_x.index.values,]
    
    ### split into train and test data ###
    k = int(n_sample * train_ratio)
    
    x_train = data_x[:k]
    x_test = data_x[k:]
    y_train = data_y[:k]
    y_test = data_y[k:]
    
    
    # ------------------------------------------------------------------------------------------------------ # 
    ### Variable Group 1: ['year', 'rating', 'votes', 'popularity_TMDB', 'runtime_TMDB'] ###

    val_group1 = ['year', 'rating', 'votes', 'popularity_TMDB', 'runtime_TMDB']
    
    ## missing value imputation ##
    # use the mean from train data to fill test data 

    # 'year', 'rating', 'votes' have missing values as 'NaN', fill by column mean
    x_train.ix[x_train['year'].isnull(), 'year'] = x_train['year'].mean()
    x_test.ix[x_test['year'].isnull(), 'year'] = x_train['year'].mean()

    x_train.ix[x_train['rating'].isnull(), 'rating'] = x_train['rating'].mean()
    x_test.ix[x_test['rating'].isnull(), 'rating'] = x_train['rating'].mean()

    x_train.ix[x_train['votes'].isnull(), 'votes'] = x_train['votes'].mean()
    x_test.ix[x_test['votes'].isnull(), 'votes'] = x_train['votes'].mean()

    # 'popularity_TMDB', 'runtime_TMDB' have missing values as 0, replace by column mean
    x_train.ix[x_train['popularity_TMDB'] == 0, 'popularity_TMDB'] = x_train['popularity_TMDB'].mean()
    x_test.ix[x_test['runtime_TMDB'] == 0, 'runtime_TMDB'] = x_train['runtime_TMDB'].mean() 
    
    ## use x_train_new, x_test_new to record the transformed data
    x_train_group1 = x_train[val_group1]
    x_test_group1 = x_test[val_group1]

    
    # ------------------------------------------------------------------------------------------------------ # 
    ### Variables Group 2: ['title', 'plot', 'plot outline', 'overview_TMDB', 'tagline_TMDB', 'mpaa_reason'] ###

    ## combine the text in ['title', 'plot', 'plot outline', 'overview_TMDB', 'tagline_TMDB']

    x_train_text = x_train['title'].str.cat([x_train['plot'], x_train['plot outline'], 
                              x_train['overview_TMDB'], x_train['tagline_TMDB']], 
                              na_rep = " ")
    x_train_text = pd.DataFrame(x_train_text, index = x_train.index)

    x_test_text = x_test['title'].str.cat([x_test['plot'], x_train['plot outline'], 
                              x_test['overview_TMDB'], x_test['tagline_TMDB']], 
                              na_rep = " ")
    x_test_text = pd.DataFrame(x_test_text, index = x_test.index)

    ## apply text analysis on combined text and return the top 10 PCs
    x_train_group2, x_test_group2 = text_analysis(x_train_text, x_test_text,
                                             val_name = 'text', n_components = 10)

    
    # ------------------------------------------------------------------------------------------------------ # 
    ### Variables Group 3: [mpaa_reason'] ###

    ## apply text analysis on 'mpaa_reason' and return thr PCs that cover 60% variance
    x_train_mpaa = pd.DataFrame(x_train['mpaa_reason'], index = x_train.index)
    x_test_mpaa = pd.DataFrame(x_test['mpaa_reason'], index = x_test.index)
    
    x_train_group3, x_test_group3 = text_analysis(x_train_mpaa, x_test_mpaa,
                                             val_name = 'mpaa', n_components = 10)
    
    
    # ------------------------------------------------------------------------------------------------------ # 
    ### Variables Group 4: ['director', 'cast', 'production company', 'writer'] ###
    
    x_train_group4_1, x_test_group4_1 = top_features(train_feature = x_train, train_genre = y_train,
                                        test_feature = x_test, test_genre = y_test,
                                        val_name = 'director',
                                        val_n = 1)
    
    x_train_group4_2, x_test_group4_2 = top_features(train_feature = x_train, train_genre = y_train,
                                        test_feature = x_test, test_genre = y_test,
                                        val_name = 'writer',
                                        val_n = 1)

    x_train_group4_3, x_test_group4_3 = top_features_pca(train_feature = x_train, train_genre = y_train,
                                        test_feature = x_test, test_genre = y_test,
                                         val_name = 'cast',
                                         val_n = 5,
                                         pca_n = 10)

    x_train_group4_4, x_test_group4_4 = top_features_pca(train_feature = x_train, train_genre = y_train,
                                        test_feature = x_test, test_genre = y_test,
                                         val_name = 'production companies',
                                         val_n = 5,
                                         pca_n = 5)
     
    x_train_group4 = pd.concat([x_train_group4_1, x_train_group4_2,
                               x_train_group4_3, x_train_group4_4], axis = 1)
    x_test_group4 = pd.concat([x_test_group4_1, x_test_group4_2,
                               x_test_group4_3, x_test_group4_4], axis = 1)
    
    # ------------------------------------------------------------------------------------------------------ # 
    ### Variables Group 5: ['animation department', 'original music'] ###
    
    # return x_train_group5, x_test_group5
    
    x_train_group5_1, x_test_group5_1 = feature_to_count(train_feature = x_train, train_genre = y_train,
                                        test_feature = x_test, test_genre = y_test,
                                        val_name = 'animation department')

    x_train_group5_2, x_test_group5_2 = feature_to_count(train_feature = x_train, train_genre = y_train,
                                        test_feature = x_test, test_genre = y_test,
                                        val_name = 'original music')
    
    
    x_train_group5 = pd.concat([x_train_group5_1, x_train_group5_2], axis = 1)
    x_test_group5 = pd.concat([x_test_group5_1, x_test_group5_2], axis = 1)
    
    # ------------------------------------------------------------------------------------------------------ # 
    ### Combine Engineered Features ###
    x_train_new = pd.concat([x_train_group1, x_train_group2, x_train_group3, 
                             x_train_group4, x_train_group5], axis=1)
    
    x_test_new = pd.concat([x_test_group1, x_test_group2, x_test_group3, 
                             x_test_group4, x_test_group5], axis=1)
    
    return (x_train_new, x_test_new, y_train, y_test)

## Supplement: functions that are used in the above meta-function:

In [60]:
##### part of the function to apply text analysis to a data series #####
#### to transform a text paragraph to bag-of-words than to one-hot coding


#----------------------------------------------------------------------------------------------------
### Input ###
        # data: a series for text analysis

        
### Output ###
        # the transformed data in one-hot coding
#----------------------------------------------------------------------------------------------------        



def text_to_matrix(data):
    
    import pandas as pd
    import numpy as np
    import re
    import warnings
    warnings.filterwarnings("ignore")
    
    
    
    ## --------------- Bag-of-Words --------------- ##
    
    ## string to list
    import re
    col_words = []
    
    for i in range(data.shape[0]):
    
        if type(data.iloc[i,0]) == str: 
            letters_only = re.sub("[^a-zA-Z]", " " , data.iloc[i,0]) # remove non-letter
            lower_case = letters_only.lower().split()   # Convert to lower case # Split into words
            
            # avoid downloading nltk
            # from NLTK stopwords https://pythonprogramming.net/stop-words-nltk-tutorial/
            stops = {'ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 
                     'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 
                     'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 
                     's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 
                     'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 
                     'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 
                     'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 
                     'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 
                     'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 
                     'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 
                     'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 
                     'it', 'how', 'further', 'was', 'here', 'than'} 
            meaningful_words = [w for w in lower_case if not w in stops]  # Remove stop words from "words"
            
            words = ( " ".join(meaningful_words))
    
        else: words = "NA"
       
        col_words.append(words)
        
        
    
    ## list to vector
    from sklearn.feature_extraction.text import CountVectorizer

    # Initialize the "CountVectorizer" object
    vectorizer = CountVectorizer(analyzer = "word",   
                                 tokenizer = None,    
                                 preprocessor = None, 
                                 stop_words = None,   
                                 max_features = 10000)

    data_array = vectorizer.fit_transform(col_words)
    data_array = pd.DataFrame(data_array.toarray())
    data_array.columns = vectorizer.get_feature_names()
    
    return data_array
    

### Function for Variable ['title', 'plot', 'plot outline', 'overview_TMDB', 'tagline_TMDB', 'mpaa_reason']

In [58]:
##### Function to apply text analysis to a column with `colname` in the data file `filename`


#----------------------------------------------------------------------------------------------------
### Input ###
        # train: the train data in one-hot coding
        # test: the test data in one-hot coding
        # val_name:  variable name that's used in naming the columns as "val_name_PCi"
        # n_components: if value is int, the number of PCs to return
                       # if value between (0,1), the variance explained by the PCs returnd
        
### Output ###
        # data matrix of engineered feature, one for train data and one for test data
#----------------------------------------------------------------------------------------------------        


def text_analysis(train, test, val_name, n_components):
    
    ## turn each text paragraph into one-hot coding
    train_array = text_to_matrix(train)
    test_array = text_to_matrix(test)
    
    ## take the union set of words in train and text data as column
    ## words that don't show up are assigned 0
    align_column = pd.concat([train_array,test_array], axis=0).fillna(0)
    
    ## split into train and text after aligning the columns
    train_array = align_column.iloc[0:train_array.shape[0], ]
    test_array = align_column.iloc[train_array.shape[0]:, ]
    
    ## PCA
    from sklearn.decomposition import PCA
    pca = PCA(n_components = n_components)
    pca.fit(train_array)
    train_new = pd.DataFrame(pca.transform(train_array), index = train.index)
    test_new = pd.DataFrame(pca.transform(test_array), index = test.index)
    
    col_names = []
    for i in range(train_new.shape[1]):
        i_name = val_name + "_PC" + str(i+1)
        col_names.append(i_name)
        
    train_new.columns = col_names
    test_new.columns = col_names
    
    return (train_new, test_new)
    

### Function used in the following functions

In [23]:
### Function to convert to string list [u'Action', u'Adventure', u'Fantasy'] into dummy coding
## input: 
           # data = orignal data frame, 
           # val_name = name of the variable
## output: a data frame

def string_to_vector(data, val_name):
    
    from sklearn.feature_extraction.text import CountVectorizer
    
    # convert any np.nan to a string 'nan'
    data[val_name][pd.isnull(data[val_name])] = 'nan'
    
    vectorizer = CountVectorizer(analyzer = "word",   
                                             tokenizer = None,    
                                             preprocessor = None, 
                                             stop_words = None,   
                                             max_features = 50000)

    val_data = vectorizer.fit_transform(data[val_name])
    df_val = pd.DataFrame(val_data.toarray())
    df_val.columns = vectorizer.get_feature_names()
    df_val.index = data.index
    
    return df_val

### Function for Variable ['director', 'writer']

In [41]:
### Function considers a particular feature of interest (e.g. director, writer) 
### and picks out the top val_n most important value in each genre
### Input: 
# - train_feature: df with 2500 train observations, all feature columns, indexed with imdb_ids
# - train_genre: df with 2500 train observations, all genre columns, indexed with imdb_ids
# - test_feature: df with 2500 test observations, all feature columns, indexed with imdb_ids 
# - test_genre: df with 2500 test observations, all genre columns, indexed with imdb_ids
# - val_name # feature of interest, eg. director
# - val_n # number of top values to take, eg. 1
### Output:
# - train_val: df with 2500 train observations, new columns of top directors, indexed with imdb_ids
# - test_val: df with 2500 test observations, new columns of top directors, indexed with imdb_ids

def top_features(train_feature, train_genre, test_feature, test_genre,
                val_name = 'director', val_n = 1):

    # convert feature of interest into dummy variables in train set and test set
    feature_val_train = string_to_vector(train_feature, val_name)
    feature_val_test = string_to_vector(test_feature, val_name)

    # create a dataframe with columns consisting of all directors and all genres, rows are movie entries 
    feature_val_genre = pd.concat([feature_val_train, train_genre], axis = 1)
    
    # generate list of top directors in each genre 
    val_list = []
    for i in train_genre.columns:
        sum_val_in_genre = feature_val_genre.ix[feature_val_genre[i] == 1, range(feature_val_train.shape[1]-1)].sum(axis = 0)
        sum_val_in_genre_sorted = sum_val_in_genre.sort(inplace=False, ascending = False)
        for j in range(val_n):
            top_val_in_genre = sum_val_in_genre_sorted.index[j]
            val_list.append(top_val_in_genre)      

    # output dataframes of movies with new columns
    train_val = feature_val_train.ix[:, val_list] 
    test_val = feature_val_test.ix[:, val_list]
    
    # rename columns as director1, director2, etc
    col_names = []
    for i in range(train_val.shape[1]):
            i_name = val_name + str(i)
            col_names.append(i_name)
    train_val.columns = col_names
    test_val.columns = col_names

    return(train_val, test_val)

### Function for Variable ['cast','production company']

In [29]:
### Function considers a particular feature of interest (e.g. cast, production companies) 
### and picks out the top val_n most important value in each genre
### and then performs PCA, picks out the top pca_n numbers of components
### Input: 
# - train_feature: df with 2500 train observations, all feature columns, indexed with imdb_ids
# - train_genre: df with 2500 train observations, all genre columns, indexed with imdb_ids
# - test_feature: df with 2500 test observations, all feature columns, indexed with imdb_ids 
# - test_genre: df with 2500 test observations, all genre columns, indexed with imdb_ids
# - val_name # feature of interest, eg. director
# - val_n # number of top values to take, eg. 5
# - pca_n # number of pca components to retain, eg. 5
### Output:
# - train_val: df with 3000 train observations, columns as pca components, indexed with imdb_ids
# - test_val: df with 2000 test observations, columns as pca components, indexed with imdb_ids

def top_features_pca(train_feature, train_genre, test_feature, test_genre,
                     val_name = 'cast',
                     val_n = 5,
                     pca_n = 5):
    
    ## Step 1, pick top casts in each genre using top_feature function 
    train_val, test_val = top_features(train_feature = train_feature,
                    train_genre = train_genre,
                    test_feature = test_feature,
                    test_genre = test_genre,
                    val_name = val_name,
                    val_n = val_n)
    
    # replace any NA values in test set with 0 (or will have problem in pca.transform)
    test_val = test_val.fillna(value=0)

    ## Step 2, perform PCA
    from sklearn.decomposition import PCA
    
    pca = PCA(n_components = pca_n, svd_solver = "full") # keep the first pca_n PCs
    pca = pca.fit(train_val)
    train_pca = pd.DataFrame(pca.transform(train_val), index = train_val.index)
    test_pca = pd.DataFrame(pca.transform(test_val), index = test_val.index)

    ## Step 3, rename the columns as cast_PC1, cast_PC2, etc. 
    col_names = []
    for i in range(pca_n):
            i_name = val_name + "_PC" + str(i+1)
            col_names.append(i_name)
    train_pca.columns = col_names
    test_pca.columns = col_names

    return (train_pca, test_pca)

### Function for Variable ['animatino department', 'original music']

In [30]:
### Function considers a particular feature of interest (e.g. animation department, original music) 
### and counts the number of member occurance in each movie 
### Input: 
# - train_feature: df with 2500 train observations, all feature columns, indexed with imdb_ids
# - test_feature: df with 2500 test observations, all feature columns, indexed with imdb_ids 
# - val_name # feature of interest, eg. animation department
### Output:
# - train_count: df with 2500 train observations, new column of count, indexed with imdb_ids
# - test_count: df with 2500 test observations, new column of count, indexed with imdb_ids

def feature_to_count(train_feature, train_genre, test_feature, test_genre,
                val_name = 'animation department'):
    
    # convert column to number of counts of members 
    train_count = pd.DataFrame(string_to_vector(train_feature, val_name).ix[:,:-1].sum(axis=1), 
                               columns = {val_name + ' count'})
    test_count = pd.DataFrame(string_to_vector(test_feature, val_name).ix[:,:-1].sum(axis=1), 
                              columns = {val_name + ' count'})
    
    return(train_count, test_count)

---
# Part 2. Performance Metrics
@Wenshan

explain how do we evaluate our model

---
# Part 3. Benchmark Models

## 3.1 Logistic Regression
@Wehshan

## 3.2 Random Forest

## 3.3 SVM

---
# Part 4. Discussion

The notebook to submit this week should at least include:

Detailed description and implementation of two different models

Description of your performance metrics

Careful performance evaluations for both models

Visualizations of the metrics for performance evaluation

Discussion of the differences between the models, their strengths, weaknesses, etc.

Discussion of the performances you achieved, and how you might be able to improve them in the future